In [27]:
#Python Libraries
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [28]:
#PySpark Libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, split, concat, lit, regexp_replace, collect_list, concat_ws, udf, when

In [29]:
spark = SparkSession.builder.appName("processingVideoGameData").getOrCreate()

In [30]:
#Setando as variáveis de ambiente
os.environ['DB_HOST'] ='40.76.241.164'
os.environ['DB_DATABASE'] ='metabase'
os.environ['DB_USER'] ='metabase'
os.environ['DB_PASS'] ='metabase'
os.environ['DB_PORT'] ='5432'

In [31]:
#Configurando o JDBC com o database PostgreSQL
jdbcHostname = os.getenv('DB_HOST')
jdbcPort = os.getenv('DB_PORT')
jdbcDatabase = os.getenv('DB_DATABASE')
jdbcUrl = "jdbc:postgresql://{0}:{1}/{2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
properties = {"user": os.getenv('DB_USER'),"password": os.getenv('DB_PASS'),"driver": "org.postgresql.Driver"}

In [32]:
#Definindo o diretório do dataset a set processado
directoryDataset = 'data/'

In [33]:
#Função recursiva para realizar a localização dos arquivos e definir o nome de cada arquivo no dataset
def localizarArquivos(pasta, extensao):
    arquivosFunc = []
    caminhoAbsoluto = os.path.abspath(pasta)
    for pastaAtual, subPastas, arquivos  in os.walk(caminhoAbsoluto):
        arquivosFunc.extend([os.path.join(pastaAtual,arquivo) for arquivo in arquivos if arquivo.endswith(extensao)])
    return arquivosFunc

In [34]:
#Rodar a função com o diretório principal
arquivos = localizarArquivos(directoryDataset, '.csv')

['/home/jovyan/work/data/vgsales.csv']


In [23]:
for file in arquivos:
    video_game_df = spark.read.option('header',True).option('encoding','cp1252').csv(file)
    video_game_df = video_game_df.select(col('Rank').cast('integer'), 
                                             'Name', 'Platform', 'Year', 'Genre', 'Publisher', 
                                             col('NA_Sales').cast('float'), col('EU_Sales').cast('float'),
                                             col('JP_Sales').cast('float'), col('Other_Sales').cast('float'),
                                             col('Global_Sales').cast('float'))
    video_game_df.write.jdbc(url=jdbcUrl, table="metabase.test", mode="append", properties=properties)